In [43]:
import os
from pathlib import Path
import jax
import jax.numpy as jnp
import qedft
from qedft.models.networks import GlobalQiCQNN, GlobalQiQNN, GlobalQNN
from qedft.config.config import Config
from qedft.train.od.trainer import KSDFTTrainer
from horqrux.utils.operator_utils import DiffMode
from loguru import logger

# Set the default dtype as float64
jax.config.update("jax_enable_x64", True)
# set gpu device
jax.config.update("jax_platform_name", "cuda")  # "cuda" or "cpu"

# Get project path
project_path = Path(os.path.dirname(os.path.dirname(qedft.__file__)))

# Load base configuration
config = Config(config_path=project_path / 'qedft' / 'config' / 'train_config.yaml').config

# Let's change to settings to have a global functional
config['network_type'] = 'conv_dqc'
config['use_amplitude_encoding'] = True

# Configure GlobalQNN
qicnn_config = {
    "network_type": "conv_dqc",
    "wrap_self_interaction": False,
    "wrap_with_negative_transform": False,
    "use_amplitude_encoding": True,
    "n_qubits": 6,  # Number of qubits in the quantum layer
    "n_var_layers": 4,  # Number of quantum layers
    "n_layers": 4,
    "largest_kernel_width": 3,
    "max_number_conv_layers": 1,  # Limit to 1 convolutional layer
    "list_qubits_per_layer": [],
    "force_qubits_per_layer_is_kernel_width": False,
    "normalization": 1.0,
    "last_layer_type": "dense",  # Use a linear layer at the end
    "use_bias_mlp": False,
    "last_layer_features": [1],
    "diff_mode": DiffMode.AD,
    "n_shots": 0,
    "key": jax.random.PRNGKey(0)
}

# Initialize the QiCNN network
network = GlobalQNN(config_dict=qicnn_config)

num_points = 513
density = jnp.linspace(0, 1, num_points)
init_fn, apply_fn = network.build_network(grids=density)

# Initialize parameters and run inference
rng_key = qicnn_config['key']
_, params = init_fn(rng_key, input_shape=(-1, num_points, 1))
output = apply_fn(params, density)

# Print the output
print(output)

# # Initialize trainer
# trainer = KSDFTTrainer(
#     config_dict=config,
#     network=network,
#     data_path=project_path / 'data' / 'od'
# )

# # Train model
# params, loss, info = trainer.train(
#     checkpoint_save_dir=project_path / 'tests' / 'ckpts'
# )

2025-07-25 13:57:25.957 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-07-25 13:57:25.959 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-07-25 13:57:25.964 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-25 13:57:25.964 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)


list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]
[-8.46313258]


In [41]:
from qedft.models.utils import count_parameters
print("Number of parameters in the Global QNN: ", count_parameters(params))

Number of parameters in the Global QNN:  207


In [42]:
print(len(params[0]))
print(len(params[1][0].flatten()))
36 + 171

36
171


207

In [ ]:
# Just quantum gates
6 * 8 * 3

144

In [5]:
import numpy as np
from horqrux.noise import DigitalNoiseInstance, DigitalNoiseType

def calculate_noise_probabilities(t1_ns, t2_ns, gate_time_ns, readout_error):
    """
    Calculate realistic noise probabilities from IBM device parameters.
    """
    # Calculate amplitude damping probability: P = 1 - exp(-t_gate/T1)
    amplitude_damping_prob = 1.0 - np.exp(-gate_time_ns / t1_ns)

    # Calculate phase damping probability: P = 1 - exp(-t_gate/T2)
    phase_damping_prob = 1.0 - np.exp(-gate_time_ns / t2_ns)

    print("\\n===========================================")
    print("Summary of noise probabilities:")
    print(f"T1: {t1_ns} ns, T2: {t2_ns} ns, Gate time: {gate_time_ns} ns")
    print(f"Amplitude damping probability: {amplitude_damping_prob}")
    print(f"Phase damping probability: {phase_damping_prob}")
    print(f"Readout error: {readout_error}")

    return {
        "amplitude_damping": amplitude_damping_prob,
        "phase_damping": phase_damping_prob,
        "readout_error": readout_error,
    }

def create_realistic_noise_config(noise_scale=1.0, device_type="best"):
    """
    Creates realistic noise configuration based on IBM device parameters.
    """
    # IBM device parameters from the script
    device_params = {
        "best": {
            "t1_ns": 593.86,
            "t2_ns": 533.33,
            "gate_time_ns": 49.78,
            "readout_error": 0.212158,
        },
        "worst": {
            "t1_ns": 433.44,
            "t2_ns": 403.38,
            "gate_time_ns": 60.00,
            "readout_error": 0.504395,
        },
    }

    params = device_params[device_type]
    noise_probs = calculate_noise_probabilities(**params)

    # Scale noise probabilities
    scaled_probs = {
        key: min(prob * noise_scale, 1.0)  # Cap at 1.0
        for key, prob in noise_probs.items()
    }

    print(f"Scaled noise probabilities: {scaled_probs}")

    # Create noise instances for quantum gates
    quantum_noise = (
        DigitalNoiseInstance(
            DigitalNoiseType.AMPLITUDE_DAMPING,
            scaled_probs["amplitude_damping"]
        ),
        DigitalNoiseInstance(
            DigitalNoiseType.PHASE_DAMPING,
            scaled_probs["phase_damping"]
        ),
    )

    # Add measurement/sampling noise as Gaussian noise
    gaussian_noise_std = scaled_probs["readout_error"] * 0.1

    return {
        "noise": quantum_noise,
        "add_gaussian_noise_to_qnn_output": True,
        "gaussian_noise_std": gaussian_noise_std,
        "scaled_probs": scaled_probs
    }

# Test noise configuration with different scales
noise_scales = [0.0, 0.1, 0.5, 1.0, 2.0, 5.0]

print("=== Testing noise configurations ===")
for scale in noise_scales:
    print(f"\\n--- Noise scale: {scale} ---")
    noise_config = create_realistic_noise_config(scale, "best")

    if scale > 0.0:
        # Create network with noise
        network_with_noise = GlobalQNN(config_dict=qicnn_config)
        init_fn_noisy, apply_fn_noisy = network_with_noise.build_network(
            grids=density,
            noise=noise_config["noise"]
        )

        # Test inference with noise
        _, params_noisy = init_fn_noisy(rng_key, input_shape=(-1, num_points, 1))
        output_noisy = apply_fn_noisy(params_noisy, density)

        print(f"Output with noise (scale {scale}): {output_noisy}")

        # Add Gaussian noise to output if specified
        if noise_config["add_gaussian_noise_to_qnn_output"]:
            noise_key = jax.random.PRNGKey(42)
            gaussian_noise = jax.random.normal(
                noise_key,
                shape=output_noisy.shape
            ) * noise_config["gaussian_noise_std"]
            output_with_gaussian = output_noisy + gaussian_noise
            print(f"Output with Gaussian noise: {output_with_gaussian}")
    else:
        print("No noise applied (baseline)")

print("\\n=== Noise testing completed ===")

2025-07-24 18:09:56.549 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-07-24 18:09:56.550 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-07-24 18:09:56.555 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-24 18:09:56.556 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)


=== Testing noise configurations ===
\n--- Noise scale: 0.0 ---
\n===========================================
Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.0), 'phase_damping': np.float64(0.0), 'readout_error': 0.0}
No noise applied (baseline)
\n--- Noise scale: 0.1 ---
\n===========================================
Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.008040734235208813), 'phase_damping': np.float64(0.008911450699779777), 'readout_error': 0.021215800000000003}
list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]
Output with noise (

2025-07-24 18:10:05.133 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-07-24 18:10:05.134 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-07-24 18:10:05.138 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-24 18:10:05.139 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)


Output with Gaussian noise: [-8.08503814]
\n--- Noise scale: 0.5 ---
\n===========================================
Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.040203671176044065), 'phase_damping': np.float64(0.044557253498898886), 'readout_error': 0.106079}
list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-07-24 18:10:10.909 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-07-24 18:10:10.910 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-07-24 18:10:10.915 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-24 18:10:10.915 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)


Output with noise (scale 0.5): [-8.73678039]
Output with Gaussian noise: [-8.73873979]
\n--- Noise scale: 1.0 ---
\n===========================================
Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.08040734235208813), 'phase_damping': np.float64(0.08911450699779777), 'readout_error': 0.212158}
list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-07-24 18:10:16.685 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-07-24 18:10:16.686 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-07-24 18:10:16.691 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-24 18:10:16.691 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)


Output with noise (scale 1.0): [-9.49175595]
Output with Gaussian noise: [-9.49567476]
\n--- Noise scale: 2.0 ---
\n===========================================
Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.16081468470417626), 'phase_damping': np.float64(0.17822901399559554), 'readout_error': 0.424316}
list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-07-24 18:10:22.466 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-07-24 18:10:22.467 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-07-24 18:10:22.472 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-24 18:10:22.473 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)


Output with noise (scale 2.0): [-9.05171062]
Output with Gaussian noise: [-9.05954823]
\n--- Noise scale: 5.0 ---
\n===========================================
Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.40203671176044065), 'phase_damping': np.float64(0.44557253498898886), 'readout_error': 1.0}
list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]
Output with noise (scale 5.0): [-10.54928486]
Output with Gaussian noise: [-10.56775604]
\n=== Noise testing completed ===


In [6]:
# Noise scale
scale = 10

print(f"\\n--- Noise scale: {scale} ---")
noise_config = create_realistic_noise_config(scale, "best")

if scale > 0.0:
    # Create network with noise
    network_with_noise = GlobalQNN(config_dict=qicnn_config)
    init_fn_noisy, apply_fn_noisy = network_with_noise.build_network(
        grids=density,
        noise=noise_config["noise"]
    )

    # Test inference with noise
    _, params_noisy = init_fn_noisy(rng_key, input_shape=(-1, num_points, 1))
    output_noisy = apply_fn_noisy(params_noisy, density)

    print(f"Output with noise (scale {scale}): {output_noisy}")

    # Add Gaussian noise to output if specified
    if noise_config["add_gaussian_noise_to_qnn_output"]:
        noise_key = jax.random.PRNGKey(42)
        gaussian_noise = jax.random.normal(
            noise_key,
            shape=output_noisy.shape
        ) * noise_config["gaussian_noise_std"]
        output_with_gaussian = output_noisy + gaussian_noise
        print(f"Output with Gaussian noise: {output_with_gaussian}")

    # Initialize trainer
    trainer = KSDFTTrainer(
        config_dict=config,
        network=network,
        data_path=project_path / 'data' / 'od'
    )

    # Train model
    params, loss, info = trainer.train(
        checkpoint_save_dir=project_path / 'tests' / 'ckpts'
    )




2025-07-24 18:10:28.280 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-07-24 18:10:28.281 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-07-24 18:10:28.286 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-24 18:10:28.286 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)


\n--- Noise scale: 10 ---
\n===========================================
Summary of noise probabilities:
T1: 593.86 ns, T2: 533.33 ns, Gate time: 49.78 ns
Amplitude damping probability: 0.08040734235208813
Phase damping probability: 0.08911450699779777
Readout error: 0.212158
Scaled noise probabilities: {'amplitude_damping': np.float64(0.8040734235208813), 'phase_damping': np.float64(0.8911450699779777), 'readout_error': 1.0}
list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-07-24 18:10:34.118 | INFO     | qedft.train.od.trainer:__init__:58 - Initialized trainer with config: {'name': 'test', 'experiment_name': 'test', 'network_type': 'conv_dqc', 'molecule_name': 'h2', 'molecule_names': ['h2'], 'dataset1': [128, 384], 'rng': 0, 'save_plot_loss': False, 'save_every_n': 20, 'activation': 'tanh', 'n_neurons': 513, 'n_layers': 2, 'n_qubits': 9, 'n_reupload_layers': 1, 'use_rzz_parametrized_entanglers': False, 'chebychev_reuploading': False, 'add_reversed_rzz': False, 'entangling_block_type': 'alternate_linear', 'single_qubit_rotations': ['rz', 'rx', 'rz'], 'use_same_parameters': False, 'add_negative_transform': False, 'wrap_with_self_interaction_layer': False, 'wrap_with_global_functional': False, 'use_correlators_in_output': False, 'output_operators': ['Z'], 'use_bias_in_output': False, 'max_train_steps': 10000, 'factr': 1.0, 'pgtol': 1e-14, 'm': 20, 'maxfun': 20, 'maxiter': 2, 'num_iterations': 15, 'ks_iter_to_ignore': 10, 'discount_factor': 0.9, 'alpha'

Output with noise (scale 10): [-13.41383076]
Output with Gaussian noise: [-13.43230193]


2025-07-24 18:10:35.716 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-07-24 18:10:35.719 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-07-24 18:10:35.724 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-24 18:10:35.724 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)
2025-07-24 18:10:35.725 | INFO     | qedft.models.wrappers:wrap_network:205 - Global model, ensuring output is scalar (wrap_self_interaction causes global models to output (num_grids,) instead of (1,))
2025-07-24 18:10:35.726 | INFO     | qedft.train.od.trainer:train:108 - Initializing fresh parameters
2025-07-24 18:10:35.728 | INFO     | qedft.train.od.trainer:train:114 - Jitting n

list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-07-24 18:10:35.951 | INFO     | qedft.train.od.trainer:train:140 - Jitting loss_fn
2025-07-24 18:10:35.953 | INFO     | qedft.train.od.trainer:train:151 - Checkpoint save directory: /home/isokolov/qex/tests/ckpts
2025-07-24 18:10:35.953 | INFO     | qedft.train.od.trainer:train:173 - Optimizing with L-BFGS-B: {'maxfun': 20, 'factr': 1.0, 'm': 20, 'pgtol': 1e-14, 'maxiter': 2}
2025-07-24 18:10:35.960 | INFO     | qedft.train.od.train:_kohn_sham:116 - Jitting kohn_sham_func
2025-07-24 18:16:16.302 | INFO     | qedft.train.od.train:np_value_and_grad_fn:454 - step 0, loss 0.4996592183852752 in 340.3263998031616 sec
2025-07-24 18:16:16.305 | INFO     | qedft.train.od.train:np_value_and_grad_fn:458 - Save checkpoint /home/isokolov/qex/tests/ckpts/ckpt-00000
2025-07-24 18:16:18.468 | INFO     | qedft.train.od.train:np_value_and_grad_fn:454 - step 1, loss 54.41394066219689 in 2.040738821029663 sec
2025-07-24 18:16:20.535 | INFO     | qedft.train.od.train:np_value_and_grad_fn:454 - step 2,